In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [2]:
pd.set_option('display.max_rows', None)
df1 = pd.read_csv("Desktop/Data/result.csv")
df1.head()

,ticker,symbolFA,nameFA,industryIndex,market
0,IRO1PAKS0001,شپاکسا,پاکسان‌,IREXCLEANINGPRODUCT,11.0
1,IRO1PKER0001,کرماشا,صنایع پتروشیمی کرمانشاه,IREXAGRICULTURALCHEMICALS,11.0
2,IRO1ABAD0001,ثاباد,توریستی ورفاهی آبادگران ایران,IREXCONSTRUCTION,15.0
3,IRO1ABDI0001,دعبید,لابراتوارداروسازی‌ دکترعبیدی‌,IREXPHARMACEUTICALS,15.0
4,IRO1ABIK0001,سابیک,سیمان آبیک,IREXOTHERNONMETALLICPRODUCTS,15.0


In [3]:
df2 = pd.read_csv("Desktop/Data/betasahm1.csv", header = None)
df2.columns = ["Date", "Post"]
df2.head(10)

,Date,Post
0,2020-04-24 04:41:28+00:00,موافقت اروپا با ایجاد صندوق اضطراری یک تریلیون...
1,2020-04-23 19:21:01+00:00,مشخصات دارایی های کچاد
2,2020-04-23 19:20:10+00:00,کچاد و عرضه فولاد غدیر ایرانیان
3,2020-04-23 19:19:01+00:00,#کچاد و نرخ محاسبه حق انتفاع
4,2020-04-23 19:15:08+00:00,میانگین نرخ دلار در بازار ثانویه از ابتدای سال...
5,2020-04-23 19:02:01+00:00,نگاهی به وضعیت فروش #سخوز
6,2020-04-23 18:48:01+00:00,دعوت به مجمع در #کفرا جلسه مجمع عمومی این شرک...
7,2020-04-23 18:41:01+00:00,نگاهی به وضعیت فروش #کنور
8,2020-04-23 18:32:01+00:00,#غمینو جلسه مجمع عمومی این شرکت در ساعت 09:00...
9,2020-04-23 18:21:01+00:00,#غمهرا سس فروش قصه ما در فروردین با فروش 247 م...


In [4]:
#extract hashtags world
hashtags = df2.Post.str.findall(r'#.*?(?=\s|$)')

In [5]:
df2['hashtags'] = hashtags

In [6]:
df2.head()

,Date,Post,hashtags
0,2020-04-24 04:41:28+00:00,موافقت اروپا با ایجاد صندوق اضطراری یک تریلیون...,[]
1,2020-04-23 19:21:01+00:00,مشخصات دارایی های کچاد,[]
2,2020-04-23 19:20:10+00:00,کچاد و عرضه فولاد غدیر ایرانیان,[]
3,2020-04-23 19:19:01+00:00,#کچاد و نرخ محاسبه حق انتفاع,[#کچاد]
4,2020-04-23 19:15:08+00:00,میانگین نرخ دلار در بازار ثانویه از ابتدای سال...,[]


In [7]:
hashtags_list_df = df2.loc[
                       df2.hashtags.apply(
                           lambda hashtags_list: hashtags_list !=[]
                       ),['hashtags']]

In [8]:
df2["hashtags"] = hashtags_list_df
df2.head()

,Date,Post,hashtags
0,2020-04-24 04:41:28+00:00,موافقت اروپا با ایجاد صندوق اضطراری یک تریلیون...,NaN
1,2020-04-23 19:21:01+00:00,مشخصات دارایی های کچاد,NaN
2,2020-04-23 19:20:10+00:00,کچاد و عرضه فولاد غدیر ایرانیان,NaN
3,2020-04-23 19:19:01+00:00,#کچاد و نرخ محاسبه حق انتفاع,[#کچاد]
4,2020-04-23 19:15:08+00:00,میانگین نرخ دلار در بازار ثانویه از ابتدای سال...,NaN


In [9]:
df = df2.dropna()
df.head()

,Date,Post,hashtags
3,2020-04-23 19:19:01+00:00,#کچاد و نرخ محاسبه حق انتفاع,[#کچاد]
5,2020-04-23 19:02:01+00:00,نگاهی به وضعیت فروش #سخوز,[#سخوز]
6,2020-04-23 18:48:01+00:00,دعوت به مجمع در #کفرا جلسه مجمع عمومی این شرک...,[#کفرا]
7,2020-04-23 18:41:01+00:00,نگاهی به وضعیت فروش #کنور,[#کنور]
8,2020-04-23 18:32:01+00:00,#غمینو جلسه مجمع عمومی این شرکت در ساعت 09:00...,[#غمینو]


In [10]:
df = df[['Date', 'hashtags', 'Post']]
df.head()

,Date,hashtags,Post
3,2020-04-23 19:19:01+00:00,[#کچاد],#کچاد و نرخ محاسبه حق انتفاع
5,2020-04-23 19:02:01+00:00,[#سخوز],نگاهی به وضعیت فروش #سخوز
6,2020-04-23 18:48:01+00:00,[#کفرا],دعوت به مجمع در #کفرا جلسه مجمع عمومی این شرک...
7,2020-04-23 18:41:01+00:00,[#کنور],نگاهی به وضعیت فروش #کنور
8,2020-04-23 18:32:01+00:00,[#غمینو],#غمینو جلسه مجمع عمومی این شرکت در ساعت 09:00...


In [11]:
pd.set_option("max_colwidth", 2)
df[["Date","hashtags", "Post"]].head(10)

,Date,hashtags,Post
3,2020-04-23 19:19:01+00:00,[#کچاد],#کچاد و نرخ محاسبه حق انتفاع
5,2020-04-23 19:02:01+00:00,[#سخوز],نگاهی به وضعیت فروش #سخوز
6,2020-04-23 18:48:01+00:00,[#کفرا],دعوت به مجمع در #کفرا جلسه مجمع عمومی این شرکت در ساعت 10:00 روز سه شنبه مورخ 1399/02/30 در استان تهران ،شهر تهران به آدرس خيابان ولي عصر بالاتر ازجام جم نرسيده به تقاطع ولي عصر چمران (چهار راه پارک وي) مجموعه فرهنگي ورزشي تلاش برگزار میگردد
7,2020-04-23 18:41:01+00:00,[#کنور],نگاهی به وضعیت فروش #کنور
8,2020-04-23 18:32:01+00:00,[#غمینو],#غمینو جلسه مجمع عمومی این شرکت در ساعت 09:00 روز چهار شنبه مورخ 1399/02/31 در استان تهران ،شهر تهران به آدرس تهران - کيلومتر 10 جاده مخصوص کرج - شرکت مينو - سالن اجتماعات شرکت دارويي مينو برگزار میگردد
9,2020-04-23 18:21:01+00:00,[#غمهرا],"#غمهرا سس فروش قصه ما در فروردین با فروش 247 میلیارد ریالی مواجه بود نرخ فروش نیز ۱۴۳,۴۸۷,۶۲۵ در اسفند نرخ فروش ۱۲۴,۵۷۷,۴۴۱ ریال بوده که رشد داشته است \nسهم در اسفند 401 میلیارد ریال فروش داشت که بیشتر به خاطر خرید های قبل عید مردم و پیتزا و ساندویچ پزی در خانه بود !!"
10,2020-04-23 18:15:10+00:00,[#غسالم],افزایش سرمایه 1395% #غسالم از محل مازاد تجدید ارزیابی دارایی ها در مجمع شرکت تصویب شد . به نظر تا سرمایه ثبت نشود نماد بازگشایی نخواهد شد .
11,2020-04-23 18:08:01+00:00,[#خکار],#خکار فقط 15 میلیارد ریال فروش داشته و البته توضیح نوشته است که با توجه به تعطیلات ۲ هفته ای ایام نوروز و همچنین محدودیت های بوجود آمده در کشور به دلیل شیوع ویروس کرونا، فعالیت شرکت از تاریخ ۱۳۹۹/۰۱/۲۳ شروع شده است.
13,2020-04-23 17:52:01+00:00,[#فملی],نگاهی به وضعیت فروش #فملی
14,2020-04-23 17:48:01+00:00,[#خشرق],#خشرق در فروردین با فروش 228 میلیارد ریالی مواجه بود و نرخ فروش نیز به نظر رشد کرده است . در مدت مشابه سال قبل فروش 305 میلیارد ریال بوده است . در سال قبل بهمن 593 میلیارد ریال و اسفند 407 میلیارد ریال فروش داشته .


In [12]:
foo = lambda a: "/".join(a) 

In [13]:
#merge posts with same hashtags worlds 
df = df.groupby(by=df.hashtags.astype(str)).agg({'Post': foo}).reset_index()
df.head()

hashtags  \
0  ['#']                 
1  ['#NAV', '#تاصیکو']   
2  ['#NAV', '#وسپه']     
3  ['#NAV', '#پارسان']   
4  ['#آ_س_پ']            

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

In [14]:
df = df.drop([0])

In [15]:
# Apply a first round of cleaning
import re
import string

def clean_text_round1(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', str(text))
    text = re.sub('[%s]' % re.escape(string.punctuation), '', str(text))
    text = re.sub('\w*\d\w*', '', str(text))
    return text

round1 = lambda x: clean_text_round1(x) 

In [16]:
df.Post = df.Post.astype(str)

In [17]:
data_clean = pd.DataFrame(df.Post.apply(round1))
data_clean.head()

,Post
1,به روزرسانی nav هر سهم تاصیکو بعد از افزایش سرمایه فولاد لازم به توضیح است که تاصیکو به صورت مستقیم مالکیت درصد از فولاد مبارکه را دارد و این سهم، دومین سهم در پرتفوی بورسی تاصیکو محسوب می شود\n در حال حاضر نسبت pnav سهم در محدوده درصد استکانال تحلیلی پارسیس
2,به روزرسانی nav هر سهم وسپه بعد از ارائه گزارش آذر ماه در جدول فوق، nav مقایسه ای آذر و دی ماه قابل مشاهده است در این روز، با وجود رشد درصدی ان ای وی، قیمت سهم درصد رشد داشته است\nدر حال حاضر نسبت pnav سهم در محدوده درصد استکانال تحلیلی پارسیس
3,به روز رسانی nav هر سهم پارسان\nدر حال حاضر نسبت pnav سهم در محدوده درصد استکانال تحلیلی پارسیس
4,پیشرفت پروژه ها در آسپآسپ در به زیان ریالی رسیده است البته این عدد با سرمایه قدیم ثبت شده استآسپ مهلت استفاده از حق تقدم خرید سهام از تاریخ لغایت تاریخ تمدید شدشفاف سازی در آسپآسپ در گزارش اسفند میلیارد ریال فروش داشته و جمع درامد ماهه میلیارد ریال شده استآسپ فسخ قرارداد شماره با موضوع مدیریت فنی و اجرایی و مالی باقیمانده عملیات اجرایی برج پروژه رویای کیش به استثنای نما فی مابین شرکت آ س پ و شرکت تیسا کیشافزایش سرمایه آسپ در جلسه هیئت مدیره مورد تصویب قرار گرفتافزایش سرمایه در آسپ با عنایت به مجوز افزایش سرمایه به شماره مجوز مورخ و باستناد مصوبۀ مجمع عمومی فوق‌العاده مورخ ، جلسۀ هیئت‌ مدیره به منظور تصمیم گیری در خصوص افزایش سرمایه شرکت از مبلغ میلیون ریال به مبلغ \n میلیون ریال از محل مطالبات و آورده نقدی مبلغ میلیون ریال ، ، در روز یکشنبه مورخ برگزار خواهد گردیدآسپ و صدور مجوز افزایش سرمایه از محل مطالبات حال شده سهامداران و آورده نقدی،آسپ و وضعیت پروژه ها \nسهم در توضیح اورده لازم به توضيح مي باشد طبق قرارداد پيمان مديريت شماره با موضوع مديريت فني ، اجرايي و مالي باقيمانده عمليات ساختماني ابنيه ، تأسيسات مکانيکي و برقي به استثناي نما پروژه برج مسکوني روياي کيش في مابين شرکت آسپ و شرکت تيسا کيش در مرحله تجهيز کارگاه مي باشد آسپ با توجه به عدم توجیه اقتصادی قرارداد مذکور و تغییر شرایط اقتصادی کشور و کاهش بازده پروژه های تجاری به جهت رکود اقتصادی حاکم و عرضه بیش از حد پروژه های تجاری در شهر تهران ، در تاریخ با موافقت طرفین قرارداد طی صورتجلسه فصل مشترک و تحویل و تحول ، ضمن خاتمه قرارداد مذکور هزینه های انجام شده پروژه توسط شرکت به مبلغ ، میلیون ریال محقق شده و مقرر گردید شرکت سمت و سوی ایرانیان مبلغ ، میلیون ریال را به شرکت آسپ پرداخت نماید و الباقی هزینه های پذیرفته نشده توسط شریک به مبلغ ، میلیون ریال در تعدیلات سنواتی منظور خواهد شد شفاف سازی آسپ در گزارش ماهانه لازم به توضيح مي باشد طبق قرارداد پيمان مديريت شماره با موضوع مديريت فني ، اجرايي و مالي باقيمانده عمليات ساختماني ابنيه ، تأسيسات مکانيکي و برقي به استثناي نما پروژه برج مسکوني روياي کيش في مابين شرکت آسپ و شرکت تيسا کيش در مرحله تجهيز کارگاه مي باشد آسپ و افزایش سرمایه به استحضار می رساند پیرو تصمیمات مجمع عمومی فوق العاده\nمورخ مبنی بر تصویب افزایش سرمایه از مبلغ ریال به مبلغ ریال از محل مطالبات حال شده سهامداران و آورده نقدی به منظور اصلاح ساختار مالي ، هیئت مدیره در نظر دارد نسبت به انجام مرحله دوم افزایش سرمایه از مبلغ ریال به مبلغ ریال از محل مطالبات حال شده سهامداران و آورده نقدی به منظور اصلاح ساختار مالي اقدام نمایدآسپ و قرارداد حدود میلیارد تومانی مدیریت فنی ، اجرایی و مالی باقیمانده عملیات ساختمانی ابنیه ، تأسیسات مکانیکی و برقی به استثنای نما برج پروژه مجتمع مسکونی رویای کیش شامل طبقه با زیربنای تقریبی متر مربع واقع در جزیره کیشآسپ و دعوت به مجمع جلسه مجمع عمومی این شرکت در ساعت روز شنبه مورخ درمحل تالار قصر شيرين واقع در تهران خيابان وليعصر عج خيابان شهيد عباسپور توانير سابق پلاک برگزار میگرددثبت افزایش سرمایه در آسپ با عنایت به مجوز افزایش سرمایه به شماره مجوز مورخ بر اساس تصمیمات مجمع عمومی فوق العاده مورخ افزایش سرمایۀ شرکت از مبلغ میلیون ریال به‌مبلغ میلیون ریال از محل مطالبات و آورده نقدی مبلغ میلیون ریال ، در تاریخ در مرجع ثبت شرکت ها به ثبت رسیده استآسپ در گزارش ماهه توفیق خاصی نداشته و ریال محقق کرده است باید دید در چه میکند آسپ در خرداد ماه با فروش میلیارد ریالی مواجه بوده و جمعا تا کنون میلیارد ریال فروش داشته استآسپ در ماهه ریال محقق کرده است باید به گزارش ماهه سهم توجه کردآسپ و شفاف سازی و قرارداد مشارکت در احداث مجتمع ایستگاهی و چند منظوره الوند سهم شرکت سهم شریک موضوع این قرارداد مشارکت در احداث مجتمع ایستگاهی و چند منظوره الوند با کاربر

In [18]:
# Apply a second round of cleaning
def clean_text_round2(text):
    '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    return text

round2 = lambda x: clean_text_round2(x)

In [19]:
data_clean["hashtags"] = df.hashtags

In [20]:
df = data_clean[['hashtags', 'Post']]
df.head()

,hashtags,Post
1,"['#NAV', '#تاصیکو']",به روزرسانی nav هر سهم تاصیکو بعد از افزایش سرمایه فولاد لازم به توضیح است که تاصیکو به صورت مستقیم مالکیت درصد از فولاد مبارکه را دارد و این سهم، دومین سهم در پرتفوی بورسی تاصیکو محسوب می شود\n در حال حاضر نسبت pnav سهم در محدوده درصد استکانال تحلیلی پارسیس
2,"['#NAV', '#وسپه']",به روزرسانی nav هر سهم وسپه بعد از ارائه گزارش آذر ماه در جدول فوق، nav مقایسه ای آذر و دی ماه قابل مشاهده است در این روز، با وجود رشد درصدی ان ای وی، قیمت سهم درصد رشد داشته است\nدر حال حاضر نسبت pnav سهم در محدوده درصد استکانال تحلیلی پارسیس
3,"['#NAV', '#پارسان']",به روز رسانی nav هر سهم پارسان\nدر حال حاضر نسبت pnav سهم در محدوده درصد استکانال تحلیلی پارسیس
4,['#آ_س_پ'],پیشرفت پروژه ها در آسپآسپ در به زیان ریالی رسیده است البته این عدد با سرمایه قدیم ثبت شده استآسپ مهلت استفاده از حق تقدم خرید سهام از تاریخ لغایت تاریخ تمدید شدشفاف سازی در آسپآسپ در گزارش اسفند میلیارد ریال فروش داشته و جمع درامد ماهه میلیارد ریال شده استآسپ فسخ قرارداد شماره با موضوع مدیریت فنی و اجرایی و مالی باقیمانده عملیات اجرایی برج پروژه رویای کیش به استثنای نما فی مابین شرکت آ س پ و شرکت تیسا کیشافزایش سرمایه آسپ در جلسه هیئت مدیره مورد تصویب قرار گرفتافزایش سرمایه در آسپ با عنایت به مجوز افزایش سرمایه به شماره مجوز مورخ و باستناد مصوبۀ مجمع عمومی فوق‌العاده مورخ ، جلسۀ هیئت‌ مدیره به منظور تصمیم گیری در خصوص افزایش سرمایه شرکت از مبلغ میلیون ریال به مبلغ \n میلیون ریال از محل مطالبات و آورده نقدی مبلغ میلیون ریال ، ، در روز یکشنبه مورخ برگزار خواهد گردیدآسپ و صدور مجوز افزایش سرمایه از محل مطالبات حال شده سهامداران و آورده نقدی،آسپ و وضعیت پروژه ها \nسهم در توضیح اورده لازم به توضيح مي باشد طبق قرارداد پيمان مديريت شماره با موضوع مديريت فني ، اجرايي و مالي باقيمانده عمليات ساختماني ابنيه ، تأسيسات مکانيکي و برقي به استثناي نما پروژه برج مسکوني روياي کيش في مابين شرکت آسپ و شرکت تيسا کيش در مرحله تجهيز کارگاه مي باشد آسپ با توجه به عدم توجیه اقتصادی قرارداد مذکور و تغییر شرایط اقتصادی کشور و کاهش بازده پروژه های تجاری به جهت رکود اقتصادی حاکم و عرضه بیش از حد پروژه های تجاری در شهر تهران ، در تاریخ با موافقت طرفین قرارداد طی صورتجلسه فصل مشترک و تحویل و تحول ، ضمن خاتمه قرارداد مذکور هزینه های انجام شده پروژه توسط شرکت به مبلغ ، میلیون ریال محقق شده و مقرر گردید شرکت سمت و سوی ایرانیان مبلغ ، میلیون ریال را به شرکت آسپ پرداخت نماید و الباقی هزینه های پذیرفته نشده توسط شریک به مبلغ ، میلیون ریال در تعدیلات سنواتی منظور خواهد شد شفاف سازی آسپ در گزارش ماهانه لازم به توضيح مي باشد طبق قرارداد پيمان مديريت شماره با موضوع مديريت فني ، اجرايي و مالي باقيمانده عمليات ساختماني ابنيه ، تأسيسات مکانيکي و برقي به استثناي نما پروژه برج مسکوني روياي کيش في مابين شرکت آسپ و شرکت تيسا کيش در مرحله تجهيز کارگاه مي باشد آسپ و افزایش سرمایه به استحضار می رساند پیرو تصمیمات مجمع عمومی فوق العاده\nمورخ مبنی بر تصویب افزایش سرمایه از مبلغ ریال به مبلغ ریال از محل مطالبات حال شده سهامداران و آورده نقدی به منظور اصلاح ساختار مالي ، هیئت مدیره در نظر دارد نسبت به انجام مرحله دوم افزایش سرمایه از مبلغ ریال به مبلغ ریال از محل مطالبات حال شده سهامداران و آورده نقدی به منظور اصلاح ساختار مالي اقدام نمایدآسپ و قرارداد حدود میلیارد تومانی مدیریت فنی ، اجرایی و مالی باقیمانده عملیات ساختمانی ابنیه ، تأسیسات مکانیکی و برقی به استثنای نما برج پروژه مجتمع مسکونی رویای کیش شامل طبقه با زیربنای تقریبی متر مربع واقع در جزیره کیشآسپ و دعوت به مجمع جلسه مجمع عمومی این شرکت در ساعت روز شنبه مورخ درمحل تالار قصر شيرين واقع در تهران خيابان وليعصر عج خيابان شهيد عباسپور توانير سابق پلاک برگزار میگرددثبت افزایش سرمایه در آسپ با عنایت به مجوز افزایش سرمایه به شماره مجوز مورخ بر اساس تصمیمات مجمع عمومی فوق العاده مورخ افزایش سرمایۀ شرکت از مبلغ میلیون ریال به‌مبلغ میلیون ریال از محل مطالبات و آورده نقدی مبلغ میلیون ریال ، در تاریخ در مرجع ثبت شرکت ها به ثبت رسیده استآسپ در گزارش ماهه توفیق خاصی نداشته و ریال محقق کرده است باید دید در چه میکند آسپ در خرداد ماه با فروش میلیارد ریالی مواجه بوده و جمعا تا کنون میلیارد ریال فروش داشته استآسپ در ماهه ریال محقق کرده است باید به گزارش ماهه سهم توجه کردآسپ و شفاف سازی و قرارداد مشارکت در احداث مجتمع ایستگاهی و چند منظوره الوند سهم شرکت س

In [21]:
# Let's pickle it for later use
df.to_pickle("corpus.pkl")

In [22]:
# We are going to create a document-term matrix using CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer()
data_cv = cv.fit_transform(data_clean.Post)
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_dtm.index = data_clean.hashtags
data_dtm.head()

,abs,absrubber,accenture,account,additive,alliance,api,apiتختال,apn,asd,...,ﻣﻮﺳﺴﻪ,ﻣﻮﺿﻮع,ﻧﭙﺬﯾﺮﻓﺖ,ﻧﺤﻮه,ﻧﺴﺒﺖ,ﻧﻤﻮد,ﻫﯿﭽﮕﻮﻧﻪ,ﻫﺎي,ﻫﺰﯾﻨﻪ,ﻻزم
hashtags,,,,,,,,,,,,,,,,,,,,,
"['#NAV', '#تاصیکو']",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"['#NAV', '#وسپه']",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"['#NAV', '#پارسان']",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
['#آ_س_پ'],0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
['#آبین'],0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
# Let's pickle it for later use
data_dtm.to_pickle("dtm.pkl")

In [24]:
# Let's also pickle the cleaned data (before we put it in document-term matrix format) and the CountVectorizer object

import pickle
df.to_pickle('data_clean.pkl')
pickle.dump(cv, open("cv.pkl", "wb"))

# reference
https://github.com/adashofdata/nlp-in-python-tutorial